In [2]:
import quandl
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split



In [10]:
#https://medium.com/@randerson112358/predict-stock-prices-using-python-machine-learning-53aa024da20a
# Get the stock data

quandl.ApiConfig.api_key = 'qKYyca8_q3vn5ws7FpwZ'

df = quandl.get("WIKI/GOOGL", trim_start = "2017-03-15", trim_end ="2018-03-15")

# Take a look at the data
#df.head()

#df = quandl.get_table('WIKI/PRICES', ticker = ['AAPL'], 
#                        qopts = { 'columns': ['date', 'adj_close'] }, 
#                       date = { 'gte': '2012-03-15', 'lte': '2018-03-15' }, 
#                        paginate=True)
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2017-03-15,867.94,869.88,861.295,868.39,1332885.0,0.0,1.0,867.94,869.88,861.295,868.39,1332885.0
2017-03-16,870.53,872.71,867.520,870.00,1104452.0,0.0,1.0,870.53,872.71,867.520,870.00,1104452.0
2017-03-17,873.68,874.42,868.370,872.37,1868252.0,0.0,1.0,873.68,874.42,868.370,872.37,1868252.0
2017-03-20,869.48,870.34,864.665,867.91,1542199.0,0.0,1.0,869.48,870.34,864.665,867.91,1542199.0
2017-03-21,870.06,873.47,847.687,850.14,2537978.0,0.0,1.0,870.06,873.47,847.687,850.14,2537978.0


In [11]:
#df.set_index('date', inplace=True)
#df.head()

In [12]:
# Get the Adjusted Close Price 
df = df[['Adj. Close']] 
# Take a look at the new data 
df.head()

,Adj. Close
Date,
2017-03-15,868.39
2017-03-16,870.00
2017-03-17,872.37
2017-03-20,867.91
2017-03-21,850.14


In [14]:
# A variable for predicting 'n' days out into the future
forecast_out = 10
#'n=30' days
#Create another column (the target ) shifted 'n' units up
df['prediction'] = df[['Adj. Close']].shift(-forecast_out)
#print the 12)
df.tail(15)

,Adj. Close,prediction
Date,,
2018-02-23,1128.09,1160.84
2018-02-26,1143.70,1165.93
2018-02-27,1117.51,1139.91
2018-02-28,1103.92,1148.89
2018-03-01,1071.41,1150.61
2018-03-02,1084.14,NaN
2018-03-05,1094.76,NaN
2018-03-06,1100.90,NaN
2018-03-07,1115.04,NaN


In [126]:
#df1 = df.reset_index()

In [127]:
#df1.head()

In [128]:
#X = df1[['Date','Adj. Close']].values

#print(X)

In [129]:

### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['prediction'],1))

#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[ 868.39  ]
 [ 870.    ]
 [ 872.37  ]
 [ 867.91  ]
 [ 850.14  ]
 [ 849.8   ]
 [ 839.65  ]
 [ 835.14  ]
 [ 838.51  ]
 [ 840.63  ]
 [ 849.87  ]
 [ 849.48  ]
 [ 847.8   ]
 [ 856.75  ]
 [ 852.57  ]
 [ 848.91  ]
 [ 845.095 ]
 [ 842.1   ]
 [ 841.7   ]
 [ 839.88  ]
 [ 841.46  ]
 [ 840.18  ]
 [ 855.13  ]
 [ 853.99  ]
 [ 856.51  ]
 [ 860.08  ]
 [ 858.95  ]
 [ 878.93  ]
 [ 888.84  ]
 [ 889.14  ]
 [ 891.44  ]
 [ 924.52  ]
 [ 932.82  ]
 [ 937.09  ]
 [ 948.45  ]
 [ 954.72  ]
 [ 950.28  ]
 [ 958.69  ]
 [ 956.71  ]
 [ 954.84  ]
 [ 955.89  ]
 [ 955.14  ]
 [ 959.22  ]
 [ 964.61  ]
 [ 942.17  ]
 [ 950.5   ]
 [ 954.65  ]
 [ 964.07  ]
 [ 970.55  ]
 [ 977.61  ]
 [ 991.86  ]
 [ 993.27  ]
 [ 996.17  ]
 [ 987.09  ]
 [ 988.29  ]
 [ 996.12  ]
 [1003.88  ]
 [ 996.68  ]
 [1001.59  ]
 [1004.28  ]
 [ 970.12  ]
 [ 961.81  ]
 [ 970.5   ]
 [ 967.93  ]
 [ 960.18  ]
 [ 958.62  ]
 [ 975.22  ]
 [ 968.99  ]
 [ 978.59  ]
 [ 976.62  ]
 [ 986.09  ]
 [ 972.09  ]
 [ 948.09  ]
 [ 961.01  ]
 [ 937.82  ]
 [ 929.68  ]
 [ 919.46  ]

In [130]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(df['prediction'])
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[ 872.37    867.91    850.14    849.8     839.65    835.14    838.51
  840.63    849.87    849.48    847.8     856.75    852.57    848.91
  845.095   842.1     841.7     839.88    841.46    840.18    855.13
  853.99    856.51    860.08    858.95    878.93    888.84    889.14
  891.44    924.52    932.82    937.09    948.45    954.72    950.28
  958.69    956.71    954.84    955.89    955.14    959.22    964.61
  942.17    950.5     954.65    964.07    970.55    977.61    991.86
  993.27    996.17    987.09    988.29    996.12   1003.88    996.68
 1001.59   1004.28    970.12    961.81    970.5     967.93    960.18
  958.62    975.22    968.99    978.59    976.62    986.09    972.09
  948.09    961.01    937.82    929.68    919.46    932.26    927.69
  940.81    951.      953.53    967.66    968.7175  976.91    975.96
  986.95    992.77    992.19    993.84    998.31    969.03    965.31
  952.51    958.33    945.5     946.56    947.64    940.3     945.79
  945.75    944.19    940.08    92

In [131]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [132]:
# Create and train the Support Vector Machine (Regressor) 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1) 
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [133]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.8751575675229127


In [134]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [135]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9377298960272669


In [136]:
# Set x_forecast equal to the last 'n' rows of the original data set from adj_close column
x_forecast = np.array(df.drop(['prediction'],1))[-forecast_out:]
print(x_forecast)

[[1148.89]
 [1150.61]]


In [137]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[1149.00100992 1150.69759591]


In [138]:
lr_prediction

array([1149.00100992, 1150.69759591])

In [139]:
# Print support vector regressor model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[1015.83308689 1011.59647132]


In [140]:
lr_prediction_df = pd.DataFrame({'Predicted_Price1':lr_prediction[:]})

In [141]:
lr_prediction_df

,Predicted_Price1
0,1149.001010
1,1150.697596


# For Loops Option

In [8]:
quandl.ApiConfig.api_key = 'qKYyca8_q3vn5ws7FpwZ'
quandl_stock_list = []
temp_date_list = []
temp_close_price_list = []
temp_stock_list = []
lr_prediction_list = []
svm_prediction_list = []
stock_list = ["GOOGL","AAPL"]
for x in stock_list:
    temp_stock = "WIKI/" + x
    quandl_stock_list.append(temp_stock)
    df = quandl.get(temp_stock, trim_start = "2017-03-15", trim_end ="2018-03-15")
    df = df[['Adj. Close']] 
    i = 0
    
    for index, row in df.iterrows():  
        temp_close_price = str(round(row['Adj. Close'],2))
        temp_date = str(df.index[i])
        temp_stock = x
        i = i + 1
        # print(temp_date + " " + temp_close_price)
           
        temp_date_list.append(temp_date)   
        temp_close_price_list.append(temp_close_price)
        temp_stock_list.append(temp_stock)
        
        stock_close_df = pd.DataFrame({"Stock Name": temp_stock_list})
        stock_close_df["Date"] = temp_date_list
        stock_close_df["Close Price"] = temp_close_price_list
        
        # Start predict
        forecast_out = 10
        df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
        
        # Define X data set
        X = np.array(df.drop(['Prediction'],1))
        X = X[:-forecast_out]
        
        # Define Y data set
        y = np.array(df['Prediction'])
        y = y[:-forecast_out]
        
        # Split the data into 80% training and 20% testing
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        
        # Create and train the Support Vector Machine (Regressor) 
        svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.2) 
        svr_rbf.fit(x_train, y_train)
        
        # Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
        # The best possible score is 1.0
        svm_confidence = svr_rbf.score(x_test, y_test)
        #print("svm confidence: ", svm_confidence)
        
        # Create and train the Linear Regression  Model
        lr = LinearRegression()
        # Train the model
        lr.fit(x_train, y_train)
        
        lr_confidence = lr.score(x_test, y_test)
        x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
        
        # Prediction
        lr_prediction = lr.predict(x_forecast)
        svm_prediction = svr_rbf.predict(x_forecast)
        
        lr_prediction_list.append(lr_prediction)
        svm_prediction_list.append(svm_prediction)
        
        stock_close_df["LR_Prediction"] = lr_prediction_list
        stock_close_df["SVM_Prediction"] = svm_prediction_list
        



stock_close_df



,Stock Name,Date,Close Price,LR_Prediction,SVM_Prediction
0,GOOGL,2017-03-15 00:00:00,868.39,"[1083.6159577973447, 1092.7360385816623, 1098....","[1118.3800330490758, 1125.9798427661008, 1038...."
1,GOOGL,2017-03-16 00:00:00,870.0,"[1079.3809155860213, 1088.18631400468, 1093.27...","[1118.4301430389546, 1032.3721879012053, 972.7..."
2,GOOGL,2017-03-17 00:00:00,872.37,"[1089.5796295893838, 1099.4419159676158, 1105....","[1118.159851394245, 1144.0358964321758, 967.01..."
3,GOOGL,2017-03-20 00:00:00,867.91,"[1088.6084240898185, 1098.102831602639, 1103.5...","[1117.377955337648, 1041.498946738137, 963.853..."
4,GOOGL,2017-03-21 00:00:00,850.14,"[1080.9954386641002, 1089.8921061881929, 1095....","[1118.3309621087026, 1106.4013028147997, 969.4..."
5,GOOGL,2017-03-22 00:00:00,849.8,"[1077.4428123537946, 1086.2081569793927, 1091....","[1112.7769723047913, 1041.8466168102025, 971.2..."
6,GOOGL,2017-03-23 00:00:00,839.65,"[1084.397812544585, 1093.6160828752586, 1098.9...","[1117.4996853024504, 1041.5841432825284, 965.6..."
7,GOOGL,2017-03-24 00:00:00,835.14,"[1082.1393705231585, 1091.3392034190852, 1096....","[1080.9741873541416, 1041.866020906351, 971.66..."
8,GOOGL,2017-03-27 00:00:00,838.51,"[1080.2738264859527, 1089.1466043934793, 1094....","[1118.2641442890322, 1032.2905214104733, 969.9..."
9,GOOGL,2017-03-28 00:00:00,840.63,"[1078.5793827754312, 1087.46101568512, 1092.59...","[1079.4809900534408, 1041.8638445043694, 971.6..."


In [15]:
stock_close_df.to_csv('stock_close_df1.csv')

# Example of what the dataframe should look like

In [19]:
example = pd.read_csv("EXAMPLE.csv")

In [20]:
example

,Date,GOOGL_Real_Price,GOOGL_LR_Prediction1,GOOGL_LR_Prediction2,GOOGL_LR_Prediction3,GOOGL_LR_Prediction4,GOOGL_LR_Prediction5,GOOGL_LR_Prediction6,GOOGL_LR_Prediction7,GOOGL_LR_Prediction8,GOOGL_LR_Prediction9,GOOGL_LR_Prediction10,Average_Predict_Price
0,3/1/2018,"1,071.41","1,071.41","1,071.41","1,071.41","1,071.41","1,071.41","1,071.41","1,071.41","1,071.41","1,071.41","1,071.41","1,071.41"
1,3/2/2018,"1,084.14","1,083.62","1,079.38","1,089.58","1,088.61","1,081.00","1,077.44","1,084.40","1,082.14","1,080.27","1,078.58","1,082.50"
2,3/5/2018,"1,094.76","1,092.74","1,088.19","1,099.44","1,098.10","1,089.89","1,086.21","1,093.62","1,091.34","1,089.15","1,087.46","1,091.61"
3,3/6/2018,"1,100.90","1,098.01","1,093.28","1,105.14","1,103.59","1,095.04","1,091.28","1,098.95","1,096.66","1,094.28","1,092.60","1,096.88"
4,3/7/2018,"1,115.04","1,110.15","1,105.00","1,118.27","1,116.23","1,106.88","1,102.95","1,111.22","1,108.91","1,106.09","1,104.42","1,109.01"
5,3/8/2018,"1,129.38","1,122.47","1,116.89","1,131.59","1,129.05","1,118.89","1,114.78","1,123.67","1,121.33","1,118.07","1,116.41","1,121.32"
6,3/9/2018,"1,160.84","1,149.48","1,142.98","1,160.81","1,157.18","1,145.25","1,140.75","1,150.97","1,148.58","1,144.36","1,142.72","1,148.31"
7,3/12/2018,"1,165.93","1,153.85","1,147.20","1,165.53","1,161.73","1,149.51","1,144.95","1,155.39","1,152.99","1,148.61","1,146.98","1,152.67"
8,3/13/2018,"1,139.91","1,131.51","1,125.62","1,141.37","1,138.47","1,127.72","1,123.47","1,132.81","1,130.45","1,126.87","1,125.22","1,130.35"
9,3/14/2018,"1,148.89","1,139.22","1,133.07","1,149.71","1,146.50","1,135.24","1,130.89","1,140.60","1,138.23","1,134.37","1,132.73","1,138.06"
